[View in Colaboratory](https://colab.research.google.com/github/mogamin/chainer-examples/blob/master/chainer_PotatoChips_Classification.ipynb)

In [1]:
# 
![ ! -e ./data ] && curl -L "https://rebrand.ly/dllab2018-hackathon-cv" -o data.tar.gz && gzip -d -c data.tar.gz | tar xf -

!apt-get install -y -qq libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80==4.3.0 
!pip install chainer==4.3.0
!pip install chainercv

import chainer
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [0]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer.datasets import LabeledImageDataset
from chainer.datasets import TransformDataset

In [0]:
from chainercv.transforms import resize

# 画像データの前処理変換
class ImageTransform(object):
    def __call__(self, in_data):
        img, label = in_data
        #_, H, W = img.shape
        #img = random_sized_crop(img)
        img = resize(img, (224, 224))
        #img = random_flip(img, x_random=True)
        return img, label

In [0]:
# データのロード
train = LabeledImageDataset('data/train/train_labels.txt', root='data/train/images')
train = TransformDataset(train, ImageTransform())
valid = LabeledImageDataset('data/valid/valid_labels.txt', root='data/valid/images')
valid = TransformDataset(valid, ImageTransform())

In [0]:
class PotatoMLP(chainer.Chain):
  def __init__(self, n_mid_units=5, n_class=9):
    super().__init__()
    with self.init_scope():
        self.fc1 = L.Linear(None, n_mid_units)
        self.fc2 = L.Linear(None, n_mid_units)
        self.fc3 = L.Linear(None, n_class)
  
  def __call__(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    h = F.relu(h)
    h = self.fc3(h)
    return h     

In [0]:
BATCHSIZE = 10
EPOCH=10
GPUID=0

In [0]:
# イテレータの準備
train_iter = chainer.iterators.SerialIterator(train, BATCHSIZE)
valid_iter = chainer.iterators.SerialIterator(valid, BATCHSIZE, repeat=False, shuffle=False)

In [8]:
#
mlp = PotatoMLP(224, 9)
model = L.Classifier(mlp)
if GPUID >= 0:
  model.to_gpu(GPUID)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [9]:
#
from chainer.training import extensions

updater = chainer.training.StandardUpdater(train_iter, optimizer, device=GPUID)
trainer = chainer.training.Trainer(updater, (EPOCH, 'epoch'), out='result/potato')
trainer.extend(extensions.Evaluator(valid_iter, model, device=GPUID))
trainer.extend(extensions.LogReport(trigger=(1,'epoch')))
trainer.extend(extensions.PrintReport(['epoch','main/accuracy','validation/main/accuracy','main/loss','validation/main/loss','elapsed_time']), trigger=(1,'epoch'))
trainer.extend(extensions.ProgressBar(update_interval=100))
trainer.run()

     total [..................................................]  0.47%
this epoch [##................................................]  4.75%
        50 iter, 0 epoch / 10 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.95%
this epoch [####..............................................]  9.49%
       100 iter, 0 epoch / 10 epochs
    5.8574 iters/sec. Estimated time to finish: 0:29:41.692792.
     total [..................................................]  1.42%
this epoch [#######...........................................] 14.24%
       150 iter, 0 epoch / 10 epochs
    5.7474 iters/sec. Estimated time to finish: 0:30:07.063401.
     total [..................................................]  1.90%
this epoch [#########.........................................] 18.98%
       200 iter, 0 epoch / 10 epochs
    5.7811 iters/sec. Estimated time to finish: 0:29:47.888162.
     total [#......................

Exception in main training loop: float() argument must be a string or a number, not 'JpegImageFile'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/trainer.py", line 306, in run
    update()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 149, in update
    self.update_core()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 153, in update_core
    batch = self._iterators['main'].next()
  File "/usr/local/lib/python3.6/dist-packages/chainer/iterators/serial_iterator.py", line 57, in __next__
    batch = [self.dataset[index] for index in self._order[i:i_end]]
  File "/usr/local/lib/python3.6/dist-packages/chainer/iterators/serial_iterator.py", line 57, in <listcomp>
    batch = [self.dataset[index] for index in self._order[i:i_end]]
  File "/usr/local/lib/python3.6/dist-packages/chainer/dataset/dataset_mixin.py", line 67, in __getitem__

TypeError: ignored